In [1]:
from matplotlib import pyplot as plt
import numpy as np
from scipy.integrate import quad as integral

# Population Model of Competition between cancer and T cells

Let $f_{C}(t, u)$ and $f_{I}(t, u)$ model the local densities of cancer and T cells. Then their growths are determined by:

$$
\frac{\delta}{\delta t}f_{C}(t, u) = R_{C}(t, u)f_{C}(t, u)
$$
$$
\frac{\delta}{\delta t}f_{I}(t, v) = R_{I}(t, v)f_{I}(t, v)
$$

where $R_{C}$ and $R_{I}$ model the net proliferation rates of cancer cells and T cells, respectively:

$$
R_{C}(t, v) := (\kappa_{C} - \mu_{C}\rho_{C}(t)) - \int_{V} \eta_{\theta_{I}}(|u-v|)f_{I}(t, v)dv
$$
$$
R_{I}(t, v) := [\int_{U}\eta_{\theta_{E}}(|u-v|)f_{C}(t, u)du + \kappa_{P}c_{P}(t)] - \frac{\mu_{I}}{1+\mu_{M}c_{M}(t)}\rho_{I}(t)
$$

The implementation is as follows:

### $\kappa_{C, P}$ the natural growth rate of each cell type.

In [2]:
kappa_C = 1
kappa_P = 1

### $\mu_{C, I, M}$ the natural death rate of each cell type.

In [3]:
mu_C = 0.5
mu_I = 0.5
mu_M = 1

### $c_{P, M}$ the immune proliferation boosters.

In [4]:
c_P = 1
c_M = 0

### $f_{C}(t, u)$ and $f_{I}(t, v)$ local densities of cancer or T cells.

In [5]:
f_C = lambda u: np.exp(-np.square(u-0.5)/(0.001))
f_I = lambda v: (np.exp(v)-1)/v - 1

### $\rho_{C, I}$ the global densities of cells.

In [6]:
rho_C = lambda _f_C: sum(_f_C)
rho_I = lambda _f_I: sum(_f_I)

### $\theta_{E, I}$ the interaction selectivity parameters and $\eta_{\theta_{E}, \theta_{I}}$ the interaction rates.

In [7]:
theta_E = 100000000 # leading to clonal expansion
theta_I = 100000000 # leading to cancer death

# The interaction rates that lead to cancer cell death (<-> rider waiting time)
eta_theta_E = lambda u,v: np.exp(-theta_E*np.square(np.abs(u-v)))
# The interaction rates that lead to clonal expansion (<-> driver idle time)
eta_theta_I = lambda u,v: np.exp(-theta_I*np.square(np.abs(u-v)))

###  $R_{C, I}$ the proliferation rates.

In [8]:
R_C = lambda _f_I,u,_f_C: (kappa_C - mu_C*rho_C(_f_C)) - sum([eta_theta_I(u, v) * v for v in _f_I])
R_I = lambda _f_C,v,_f_I: (sum([eta_theta_E(u, v) * u for u in _f_C])  + kappa_C*c_P) - mu_I/(1 + mu_M*c_M * rho_I(_f_I))

## Modelling the growth

To recreate the model, $f_{C,I}$ will be an array of its values where each index reprents its value at time $t$.

At every time step, $f_{C, I}[t+1] = f_{C, I}[t] + \frac{\delta}{\delta t}f_{C, I}[t] = f_{C, I}[t] + R_{C, I} * f_{C, I}[t]$

We will compute this evolution for all possible starting values of $u$ and $v$.

In [9]:
# Initial values of cancer and T cells
space = np.linspace(0, 1, num=2)
cancer = np.array([[f_C(u) for u in space]])
immune = np.array([[f_I(v) for v in space]])

# Apply the growth equation for each time step
t = 120
for _ in range(t):
    _f_C = cancer[-1]
    _f_I = immune[-1]
    
    print(f"R_C has value {np.array([R_C(_f_I, u, _f_C) for u in _f_C])}")
    print(f"R_I has value {np.array([R_I(_f_C, v, _f_I) for v in _f_I])}")
    
    delta_C = _f_C + _f_C * np.array([R_C(_f_I, u, _f_C) for u in _f_C])
    delta_I = _f_I + _f_I * np.array([R_I(_f_C, v, _f_I) for v in _f_I])
    
    np.append(cancer, delta_C)
    np.append(immune, delta_I)

R_C has value [nan nan]
R_I has value [nan nan]
R_C has value [nan nan]
R_I has value [nan nan]
R_C has value [nan nan]
R_I has value [nan nan]
R_C has value [nan nan]
R_I has value [nan nan]
R_C has value [nan nan]
R_I has value [nan nan]
R_C has value [nan nan]
R_I has value [nan nan]
R_C has value [nan nan]
R_I has value [nan nan]
R_C has value [nan nan]
R_I has value [nan nan]
R_C has value [nan nan]
R_I has value [nan nan]
R_C has value [nan nan]
R_I has value [nan nan]
R_C has value [nan nan]
R_I has value [nan nan]
R_C has value [nan nan]
R_I has value [nan nan]
R_C has value [nan nan]
R_I has value [nan nan]
R_C has value [nan nan]
R_I has value [nan nan]
R_C has value [nan nan]
R_I has value [nan nan]
R_C has value [nan nan]
R_I has value [nan nan]
R_C has value [nan nan]
R_I has value [nan nan]
R_C has value [nan nan]
R_I has value [nan nan]
R_C has value [nan nan]
R_I has value [nan nan]
R_C has value [nan nan]
R_I has value [nan nan]
R_C has value [nan nan]
R_I has value [n

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:2: RuntimeWarning: invalid value encountered in double_scalars
  
